In [1]:
import matplotlib as plt
print('matplotlib: {}'.format(plt.__version__))
# pandas
import pandas as pd
print('pandas: {}'.format(pd.__version__))
# scikit-learn
import sklearn as sk
print('sklearn: {}'.format(sk.__version__))
from pandas_profiling import ProfileReport
import seaborn as sns
print('seaborn: {}'.format(sns.__version__))
import numpy as np
print('numpy: {}'.format(np.__version__))
from sklearn import linear_model
from sklearn import svm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

matplotlib: 3.3.2
pandas: 1.1.3
sklearn: 0.23.2
seaborn: 0.11.0
numpy: 1.19.2


In [2]:
X_train = pd.read_csv("X_train.csv")

In [3]:
X_val=pd.read_csv("X_val.csv")
X_test = pd.read_csv("X_test.csv")

In [4]:
X_test.shape

(1459, 78)

In [5]:
y_train = pd.read_csv("y_train.csv")
y_val = pd.read_csv("y_val.csv")

In [86]:
id_test = pd.read_csv("id_test.csv")

In [102]:
sample_submission = pd.read_csv("sample_submission.csv")

In [105]:
id_test2 = sample_submission.Id

In [8]:
X_train.shape

(1019, 79)

In [9]:
X_train = X_train.drop(columns=["Unnamed: 0"])

In [10]:
X_val = X_val.drop(columns=["Unnamed: 0"])

In [11]:
X_test = X_test.drop(columns=["Unnamed: 0"])

In [12]:
y_train =y_train.drop(columns=["Unnamed: 0"])

In [13]:
y_val = y_val.drop(columns=["Unnamed: 0"])

In [14]:
id_test = id_test.drop(columns=["Unnamed: 0"])

In [90]:
id_test.reset_index(inplace=True)

In [93]:
id_test = id_test.drop(columns=["index"])

In [15]:
models = []
models.append(('LR', LinearRegression()))
models.append(('Ridge', linear_model.Ridge()))
models.append(('SVR', svm.SVR()))
models.append(('Lasso',linear_model.Lasso()))
models.append(('ElasticNet',ElasticNet()))
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold,scoring='neg_root_mean_squared_error')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: -0.135828 (0.008469)
Ridge: -0.134779 (0.007722)


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

SVR: -0.336435 (0.017211)
Lasso: -0.302402 (0.020140)
ElasticNet: -0.299317 (0.020669)


In [19]:
# The best model is the ridge model

In [20]:
# Let's make a baseline

In [16]:
model = linear_model.Ridge()
model.fit(X_train, y_train)
predictions = model.predict(X_val)

In [17]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_val, predictions,squared=False)

0.14740600048182884

In [38]:
# Let's tune the hyperparameters

In [20]:
from sklearn.model_selection import GridSearchCV
alphas = np.array([1,0.1,0.01,0.001,0.0001,0])
model = linear_model.Ridge()
grid = GridSearchCV(estimator=model, param_grid=dict(alpha=alphas), scoring='neg_root_mean_squared_error')
grid.fit(X_train, y_train)
print(grid.best_score_)
print(grid.best_estimator_.alpha)


-0.13415896351556014
1.0


In [ ]:
# During EDA i took the log of the sale price to have a normally distributed feature
# Now I have to undo it to submit my results.

In [36]:
X_train.shape

(1019, 78)

In [21]:
X_train2 = X_train.drop(columns="Condition2_RRAe")

In [22]:
model2 = linear_model.Ridge()
model2.fit(X_train2, y_train)

Ridge()

In [23]:
X_test.shape

(1459, 77)

In [24]:
#model.fit(X_train, y_train)
predictions2 = model2.predict(X_test)


In [25]:
predictions2.shape

(1459, 1)

In [45]:
SalePrice =  np.exp(predictions2)

In [46]:
SalePrice = pd.DataFrame(SalePrice.flatten())

In [106]:
submission = pd.concat([id_test2,SalePrice],axis=1)

In [108]:
submission = submission.rename(columns={0:"SalePrice"})

In [110]:
submission.to_csv("submission1.csv")

In [111]:
# Let's export our first model

In [112]:
import joblib
joblib.dump(model2, 'ridge_model.pkl', compress=9)

['ridge_model.pkl']

In [121]:
# Let's try a more complex model on less features

In [122]:
X_train2 = pd.read_csv("X_train2.csv")
X_val2 = pd.read_csv("X_val2.csv")
X_test2 = pd.read_csv("X_test2.csv")
y_train2 = pd.read_csv("y_train2.csv")
y_val2 = pd.read_csv("y_val2.csv")

In [132]:
X_train2 = X_train2.drop(columns=["Unnamed: 0"])
X_val2 = X_val2.drop(columns=["Unnamed: 0"])
X_test2 = X_test2.drop(columns=["Unnamed: 0"])
y_train2 = y_train2.drop(columns=["Unnamed: 0"])
y_val2 = y_val2.drop(columns=["Unnamed: 0"])

In [133]:
X_test2.shape

(1459, 13)

In [136]:
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
import xgboost as XGB
import lightgbm as lgb
from sklearn.ensemble import AdaBoostRegressor
models = []
models.append(('LR', LinearRegression()))
models.append(('Ridge', linear_model.Ridge()))
models.append(('SVR', svm.SVR()))
models.append(('Lasso',linear_model.Lasso()))
models.append(('ElasticNet',ElasticNet()))
models.append(('SGDRegressor',SGDRegressor()))
models.append(('DecisionTreeRegressor',DecisionTreeRegressor()))
models.append(('RandomForestRegressor',RandomForestRegressor()))
models.append(('KNNRegressor',KNeighborsRegressor()))
models.append(('GaussianProcessRegressor',GaussianProcessRegressor()))
models.append(('XGB',XGB.XGBRegressor()))
models.append(('LGBM',lgb.LGBMRegressor(objective='regression')))
models.append(('Adaboost',AdaBoostRegressor()))
results = []
names = []
for name, model in models:
    kfold = KFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train2, y_train2, cv=kfold,scoring='neg_root_mean_squared_error')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: -0.146406 (0.009015)
Ridge: -0.146396 (0.008850)


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

SVR: -0.332376 (0.017324)
Lasso: -0.302402 (0.020140)
ElasticNet: -0.299317 (0.020669)


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

SGDRegressor: -2857874146114284.000000 (2272205583122694.500000)
DecisionTreeRegressor: -0.202453 (0.015683)


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:531: DataConversionWarning: A column-vector y wa

RandomForestRegressor: -0.153566 (0.008527)
KNNRegressor: -0.230470 (0.014101)
GaussianProcessRegressor: -6.723774 (1.274336)
XGB: -0.166595 (0.008012)
LGBM: -0.157355 (0.008907)


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

Adaboost: -0.179476 (0.009124)


C:\Users\dalil\anaconda3\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [135]:
# Let's try the Random Forest Regressor 

In [139]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [140]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train2, y_train2)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  3.3min finished
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [141]:
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': 50,
 'bootstrap': True}

In [142]:
# Let's evaluate the best model from random search

In [145]:
model3 = RandomForestRegressor(n_estimators = 200,min_samples_split=2,min_samples_leaf=1,max_features = 'sqrt',
                              max_depth = 50,bootstrap=True)
model3.fit(X_train2, y_train2)
predictions3 = model3.predict(X_val2)

<ipython-input-145-b5add13e4154>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model3.fit(X_train2, y_train2)


In [146]:
mean_squared_error(y_val2, predictions3,squared=False)

0.16117424137580716

In [151]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [60, 70, 80, 90],
    'max_features': ['sqrt'],
    'min_samples_leaf': [1, 2, 3],
    'min_samples_split': [2, 4, 6],
    'n_estimators': [50, 100, 200, 500]
}
# Create a based model
rf = RandomForestRegressor()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [152]:
grid_search.fit(X_train2, y_train2)
grid_search.best_params_

Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   17.0s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   36.7s
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:   45.1s finished
C:\Users\dalil\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:765: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


{'bootstrap': True,
 'max_depth': 90,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}

In [153]:
model4 = RandomForestRegressor(n_estimators = 200,min_samples_split=2,min_samples_leaf=1,max_features = 'sqrt',
                              max_depth = 90,bootstrap=True)
model4.fit(X_train2, y_train2)
predictions4 = model4.predict(X_val2)

<ipython-input-153-861082e9e810>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model4.fit(X_train2, y_train2)


In [168]:
X_test2 = X_test2.drop(columns=["MSZoning_None"])

In [215]:
prediction5 = model4.predict(X_test2)

In [216]:
SalePrice3 =  np.exp(prediction5)

In [217]:
SalePrice3= pd.DataFrame(SalePrice3.flatten())
submission2 = pd.concat([id_test2,SalePrice3],axis=1)
#submission2 = submission.rename(columns={0:"SalePrice3"})

In [218]:
submission2

,Id,0
0,1461,115754.839549
1,1462,137789.456274
2,1463,173048.383038
3,1464,184994.178849
4,1465,206223.688880
...,...,...
1454,2915,80928.136649
1455,2916,82464.903853
1456,2917,153281.887148
1457,2918,103002.333023


In [221]:
submission2.to_csv("submission2.csv")

In [178]:
submission2.shape

(1459, 2)

In [179]:
submission2

,Id,SalePrice
0,1461,183685.074272
1,1462,173107.493688
2,1463,129196.004266
3,1464,159628.350295
4,1465,118541.720441
...,...,...
1454,2915,NaN
1455,2916,NaN
1456,2917,NaN
1457,2918,NaN
